In [1]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv(r'C:\Users\boyu571\boyu571_Github\01_Kakaobank_SKKU_Research_23\data\foreign_news_data\eng_news_nocontent.csv')
df

,source,author,title,description,url,urlToImage,publishedAt,content,press
0,"{'id': None, 'name': 'Trendmicro.com'}",Trend Micro,REDBALDKNIGHT/BRONZE BUTLER’s Daserf Backdoor ...,"REDBALDKNIGHT, also known as BRONZE BUTLER and...",https://blog.trendmicro.com/trendlabs-security...,https://blog.trendmicro.com/trendlabs-security...,2017-11-07T12:34:44Z,by Joey Chen and MingYen Hsieh (Threat Analyst...,Trendmicro.com
1,"{'id': None, 'name': 'Computerworld'}","Lucas Mearian, Lucas Mearian",What is blockchain? The most disruptive tech i...,Blockchain is poised to change IT in much the ...,https://www.computerworld.com/article/3191077/...,https://images.techhive.com/images/article/201...,2017-11-08T01:06:00Z,Blockchain is poised to change IT in much the ...,Computerworld
2,"{'id': None, 'name': 'hackernoon.com'}",Yaoqi Jia,Demystifying Hashgraph: Benefits and Challenges,"Recently, on our Zilliqa social channels a num...",https://hackernoon.com/demystifying-hashgraph-...,NaN,2017-11-08T17:21:35Z,"Recently, on our Zilliqa social channels a num...",hackernoon.com
3,"{'id': None, 'name': 'Topix.com'}",NaN,BitTorrent inventor announces eco-friendly Bit...,A Bitcoin transaction wastes as much electrici...,http://www.topix.com/tech/p2p/2017/11/bittorre...,http://picproxy.topixcdn.com/pics/up-,2017-11-08T23:21:17Z,A Bitcoin transaction wastes as much electrici...,Topix.com
4,"{'id': None, 'name': 'Ml-implode.com'}",NaN,Bitcoin Fork Called Off: Prices Soar After Seg...,The price of a bitcoin went on a tear on Wedne...,http://ml-implode.com/staticnews/2017-11-09_Bi...,NaN,2017-11-09T00:00:00Z,The price of a bitcoin went on a tear on Wedne...,Ml-implode.com
...,...,...,...,...,...,...,...,...,...
744427,"{'id': None, 'name': 'Biztoc.com'}",u.today,"Negatively Correlates With Bitcoin (BTC), What...",Shiba Inu continues to focus on utility as its...,https://biztoc.com/x/8248d6342c37d743,https://c.biztoc.com/p/8248d6342c37d743/s.webp,2023-09-30T21:50:10Z,Shiba Inu continues to focus on utility as its...,Biztoc.com
744428,"{'id': None, 'name': 'Slashdot.org'}",feedfeeder,Industry leaders and policymakers weigh in on ...,Industry leaders and policymakers weigh in on ...,https://slashdot.org/firehose.pl?op=view&amp;i...,NaN,2023-09-30T21:53:20Z,Sign up for the Slashdot newsletter! OR check ...,Slashdot.org
744429,"{'id': None, 'name': 'Investing.com'}",U.Today,Shiba Inu (SHIB) Negatively Correlates With Bi...,Shiba Inu (SHIB) Negatively Correlates With Bi...,https://www.investing.com/news/cryptocurrency-...,https://i-invdn-com.investing.com/news/LYNXNPE...,2023-09-30T22:00:22Z,U.Today - Dog-themed cryptocurrency is seeing ...,Investing.com
744430,"{'id': None, 'name': 'Ambcrypto.com'}",Benjamin Njiri,Can NEAR stay above $1?,"NEAR could be ripe for a price reversal, but t...",https://ambcrypto.com/?p=347645,https://statics.ambcrypto.com/wp-content/uploa...,2023-09-30T22:30:04Z,Disclaimer: The information presented does not...,Ambcrypto.com


In [333]:
df[df['press'] == 'Bangkok Post']['url'].iloc[100]

'https://www.bangkokpost.com/business/finance/1411318/institutions-asked-to-avoid-digital-tokens'

In [335]:
BangkokPost = df[df['press'] == 'Bangkok Post']
BangkokPost

,source,author,title,description,url,urlToImage,publishedAt,content,press
614,"{'id': None, 'name': 'Bangkok Post'}",Bangkok Post Public Company Limited,Foreign gangs in government crosshairs,"Almost 100,000 foreign nationals who have over...",https://www.bangkokpost.com/news/general/13594...,https://www.bangkokpost.com/media/content/2017...,2017-11-12T21:30:00Z,Even elite units such as the Special Weapons A...,Bangkok Post
1068,"{'id': None, 'name': 'Bangkok Post'}",Bangkok Post Public Company Limited,Think bitcoin's getting expensive? Try Zimbabwe,With Zimbabwe under threat of cash shortages a...,https://www.bangkokpost.com/vdo/business-and-p...,https://www.bangkokpost.com/media/content/2017...,2017-11-14T01:17:00Z,Readers are urged not to submit comments that ...,Bangkok Post
2276,"{'id': None, 'name': 'Bangkok Post'}",Bangkok Post Public Company Limited,Bitcoin emerges as crisis currency,NEW YORK: About a third of the customers queui...,https://www.bangkokpost.com/business/world/136...,https://www.bangkokpost.com/media/content/2017...,2017-11-17T08:51:00Z,A bitcoin coin is seen in an illustration pict...,Bangkok Post
2646,"{'id': None, 'name': 'Bangkok Post'}",Bangkok Post Public Company Limited,My beloved dog must suffer,My beloved dog of 15 years is dying. His desce...,https://www.bangkokpost.com/lifestyle/social-a...,https://www.bangkokpost.com/media/content/dcx/...,2017-11-18T21:00:00Z,PHOTO: 123RF My beloved dog of 15 years is dyi...,Bangkok Post
2809,"{'id': None, 'name': 'Bangkok Post'}",Bangkok Post Public Company Limited,Logistics 4.0,The fourth industrial revolution and rapid ado...,https://www.bangkokpost.com/business/news/1363...,https://www.bangkokpost.com/media/content/dcx/...,2017-11-20T05:30:00Z,The fourth industrial revolution and rapid ado...,Bangkok Post
...,...,...,...,...,...,...,...,...,...
731811,"{'id': None, 'name': 'Bangkok Post'}",Post Reporters,Cabinet works on poll pledges,The government looks set to roll out a number ...,https://www.bangkokpost.com/thailand/general/2...,https://static.bangkokpost.com/media/content/2...,2023-09-03T23:28:00Z,The government looks set to roll out a number ...,Bangkok Post
735759,"{'id': None, 'name': 'Bangkok Post'}",Wassayos Ngamkham,Three Chinese crypto fraud suspects held,Three Chinese nationals have been arrested in ...,https://www.bangkokpost.com/thailand/general/2...,https://static.bangkokpost.com/media/content/2...,2023-09-12T13:37:00Z,Three Chinese nationals have been arrested in ...,Bangkok Post
736061,"{'id': None, 'name': 'Bangkok Post'}","Wichit Chantanusornsiri, Somruedi Banchongduan...","Probing the B10,000 digital handout",The Pheu Thai Party-led coalition government b...,https://www.bangkokpost.com/business/general/2...,https://static.bangkokpost.com/media/content/d...,2023-09-12T23:14:00Z,The Pheu Thai Party-led coalition government b...,Bangkok Post
736193,"{'id': None, 'name': 'Bangkok Post'}",Reuters and Online Reporters,US jails OneCoin fraudster for 20 years,NEW YORK: A fake cryptocurrency promoter who w...,https://www.bangkokpost.com/thailand/general/2...,https://static.bangkokpost.com/media/content/2...,2023-09-13T07:36:00Z,NEW YORK: A fake cryptocurrency promoter who w...,Bangkok Post


In [282]:

url = 'https://www.entrepreneur.com/article/303556'
headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}
response = requests.get(url, headers=headers)
# soup = BeautifulSoup(response.text, "html.parser")
print(response.status_code)
# soup

200


In [339]:
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import traceback

def get_contents(yahoo):
    contents = []

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

    for url in tqdm(yahoo['url']):
        try:
            response = requests.get(url, headers=headers, timeout=60)  # 1분 timeout 설정
            soup = BeautifulSoup(response.text, "html.parser")

            # First, try to find elements with the '.article-comm div' selector
            article_elements = soup.select('.article-content')
            
            
            # # # If not found, try the '.p-m-15' selector
            if not article_elements:
                article_elements = soup.select('.boxvdo-detail')

            # # If still not found, try the '.article-text' selector
            # if not article_elements:
            #     article_elements = soup.select('.p-m-15')

            if article_elements:
                content = article_elements[0].get_text()
            else:
                content = ''
            # print(article_elements)

            contents.append(content)
        except (requests.Timeout, Exception) as e:
            if isinstance(e, requests.Timeout):
                print(f"Request timed out for URL: {url}")
            else:
                print(f"Error occurred: {e}")
                traceback.print_exc()
            contents.append('')

    return contents


In [26]:
# import requests
# from bs4 import BeautifulSoup
# import re
# from tqdm import tqdm

# def get_contents(yahoo):
#     contents = []

#     headers = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

#     for url in tqdm(yahoo['url']):
#         try:
#             response = requests.get(url, headers=headers)
#             soup = BeautifulSoup(response.text, "html.parser")

#             # 모든 클래스 이름 중에서 ".col-*-offset-1" 패턴을 가진 것들을 찾는다
#             classes = set()
#             for tag in soup.find_all(True, {'class': True}):
#                 for class_name in tag['class']:
#                     if re.match(r'col-\w+-offset-1', class_name):
#                         classes.add(class_name)

#             # 해당 패턴을 가진 클래스들로부터 내용을 추출한다
#             content = []
#             for class_name in classes:
#                 content.extend(soup.select(f".{class_name}"))
            
#             cleaned_content = re.sub('<[^>]*>', '', ''.join(str(item) for item in content))
#             contents.append(cleaned_content)
#         except:
#             contents.append('')

#     return contents


In [337]:
head = BangkokPost.head()
head['content_full'] = get_contents(head)
head

 40%|████      | 2/5 [00:25<00:42, 14.01s/it]

Request timed out for URL: https://www.bangkokpost.com/vdo/business-and-property/1360231/think-bitcoins-getting-expensive-try-zimbabwe


100%|██████████| 5/5 [01:05<00:00, 13.03s/it]
C:\Users\boyu571\AppData\Local\Temp\ipykernel_19872\3885702176.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  head['content_full'] = get_contents(head)


,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
614,"{'id': None, 'name': 'Bangkok Post'}",Bangkok Post Public Company Limited,Foreign gangs in government crosshairs,"Almost 100,000 foreign nationals who have over...",https://www.bangkokpost.com/news/general/13594...,https://www.bangkokpost.com/media/content/2017...,2017-11-12T21:30:00Z,Even elite units such as the Special Weapons A...,Bangkok Post,\n\n\n\n\n Even elite...
1068,"{'id': None, 'name': 'Bangkok Post'}",Bangkok Post Public Company Limited,Think bitcoin's getting expensive? Try Zimbabwe,With Zimbabwe under threat of cash shortages a...,https://www.bangkokpost.com/vdo/business-and-p...,https://www.bangkokpost.com/media/content/2017...,2017-11-14T01:17:00Z,Readers are urged not to submit comments that ...,Bangkok Post,
2276,"{'id': None, 'name': 'Bangkok Post'}",Bangkok Post Public Company Limited,Bitcoin emerges as crisis currency,NEW YORK: About a third of the customers queui...,https://www.bangkokpost.com/business/world/136...,https://www.bangkokpost.com/media/content/2017...,2017-11-17T08:51:00Z,A bitcoin coin is seen in an illustration pict...,Bangkok Post,\n\n\n\n\n A bitcoin ...
2646,"{'id': None, 'name': 'Bangkok Post'}",Bangkok Post Public Company Limited,My beloved dog must suffer,My beloved dog of 15 years is dying. His desce...,https://www.bangkokpost.com/lifestyle/social-a...,https://www.bangkokpost.com/media/content/dcx/...,2017-11-18T21:00:00Z,PHOTO: 123RF My beloved dog of 15 years is dyi...,Bangkok Post,\nMy beloved dog of 15 years is dying. His des...
2809,"{'id': None, 'name': 'Bangkok Post'}",Bangkok Post Public Company Limited,Logistics 4.0,The fourth industrial revolution and rapid ado...,https://www.bangkokpost.com/business/news/1363...,https://www.bangkokpost.com/media/content/dcx/...,2017-11-20T05:30:00Z,The fourth industrial revolution and rapid ado...,Bangkok Post,\nThe fourth industrial revolution and rapid a...


In [340]:
# Adding the content to the DataFrame
BangkokPost['content_full'] = get_contents(BangkokPost)
BangkokPost

  3%|▎         | 35/1227 [01:19<45:14,  2.28s/it]  


KeyboardInterrupt: 

In [285]:
head['url'].iloc[2]

'http://www.thestar.com.my/business/business-news/2017/11/13/bitcoin-tumble-erases-us$38b-as-rival-cryptocurrency-gains/'

In [323]:
head['content_full'].iloc[0]

'\n\n\n\n\n\n\n\n\n\n\n\nGamesBeat Next unites gaming industry leaders for exceptional content, networking, and deal-making opportunities. Join us on Oct 23-24 in San Francisco.\xa0 Register Now\n\n\n\n\n\n\n\n\n\n\n\n\n\nNvidia reported earnings that beat expectations and showed that the company’s focus on artificial intelligence is still paying off. For the past decade, Nvidia has been rising above graphics chips for gamers, expanding to parallel processing in data centers and lately to artificial intelligence processing for deep learning neural networks and self-driving cars.\nThe company reported earnings per share of $1.33 (up 60 percent from a year ago) on revenue of $2.6 billion (up 32 percent), beating Wall Street’s expectations.\nThe company’s stock price is up more than 100 percent in the past year on the popularity of artificial intelligence. But it slumped during the day on Thursday, along with the broader market. Analysts had expected the Santa Clara, California company to

In [325]:
empty_rows = VentureBeat[VentureBeat['content_full'] == '']
print(len(empty_rows))

5


In [308]:
empty_rows['url'].iloc[20]

'https://global.techradar.com/en-za/news/clone-trade-crypto-easily-and-safely-with-koinexpert'

<!-- # empty_rows['url'].iloc[0] -->

In [326]:
VentureBeat.to_csv(r'C:\Users\boyu571\boyu571_Github\01_Kakaobank_SKKU_Research_23\data\foreign_news_data\VentureBeat_82.csv', encoding='utf-8-sig')

In [ ]:
#article-13715 > div.post__content-wrapper > div.post-content